In [1]:
import pandas as pd
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from tqdm import tqdm  # Import tqdm for the progress bar
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.utils import filter_complex_metadata
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
def fetch_docs(title):
    try:
        # Initialize the loader for the given Wikipedia title
        loader = WikipediaLoader(query=title, load_max_docs=1, doc_content_chars_max=999999999)

        # Load returns a list of Documents
        docs = loader.load()
        docs = filter_complex_metadata(docs)

        if docs:
            return docs
        else:
            print(f"Warning: No documents found for title '{title}'.")
            return []

    except Exception as e:
        print(f"Error fetching '{title}': {e}")
        return []

In [3]:
csv_file = "../data/qid_to_titles.csv"
df = pd.read_csv(csv_file)  # The file has columns: QID, Wikipedia_Title

all_docs = []
successful_fetches = 0  # Counter for successful fetches

In [4]:
# Define the number of worker threads
MAX_WORKERS = 20  # Adjust based on your requirements and Colab's capabilities

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submit all tasks to the executor
    future_to_title = {executor.submit(fetch_docs, row["Wikipedia_Title"]): idx for idx, row in df.iterrows()}

    # Use tqdm to display the progress bar
    for future in tqdm(as_completed(future_to_title), total=len(future_to_title), desc="Fetching Wikipedia pages"):
        docs = future.result()
        if docs:
            all_docs.extend(docs)
            successful_fetches += 1

print(f"Successfully fetched {successful_fetches} documents out of {len(df)}.")

Fetching Wikipedia pages:   4%|▍         | 584/13891 [00:50<19:15, 11.52it/s]

Error fetching 'Michaela Pejzlová': HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?prop=extracts&explaintext=&exintro=&titles=Michaela+Pejzlov%C3%A1&format=json&action=query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1396b1bb0>: Failed to resolve 'en.wikipedia.org' ([Errno 8] nodename nor servname provided, or not known)"))


Fetching Wikipedia pages:   6%|▌         | 803/13891 [01:19<17:34, 12.41it/s]  /opt/miniconda3/envs/NLP/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/miniconda3/envs/NLP/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Fetching Wikipedia pages:   6%|▌         | 805/13891 [01:20<18:58, 11.49it/s]

Fetching Wikipedia pages:   7%|▋         | 914/13891 [01:27<11:55, 18.15it/s]

Error fetching 'Matúš Vallo': HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /w/api.php?prop=extracts&explaintext=&exintro=&titles=Mat%C3%BA%C5%A1+Vallo&format=json&action=query (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x13972bdf0>: Failed to resolve 'en.wikipedia.org' ([Errno 8] nodename nor servname provided, or not known)"))


...

Fetching Wikipedia pages:  89%|████████▉ | 12394/13891 [15:11<01:19, 18.87it/s]

Fetching Wikipedia pages:  92%|█████████▏| 12774/13891 [15:36<01:11, 15.53it/s]

Fetching Wikipedia pages:  99%|█████████▉| 13754/13891 [16:48<00:10, 13.33it/s]

Fetching Wikipedia pages: 100%|██████████| 13891/13891 [16:57<00:00, 13.65it/s]

Successfully fetched 13863 documents out of 13891.


In [5]:
import pickle

# After fetching all_docs
with open("../data/all_docs.pkl", "wb") as f:
    pickle.dump(all_docs, f)
    print("Successfully saved all_docs to ../data/all_docs.pkl")

Successfully saved all_docs to ../data/all_docs.pkl
